OLD original file , dont use

In [10]:
# !pip install nltk

In [1]:
import numpy as np
import pandas as pd
import re
import random
import nltk
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
lemmatizer = WordNetLemmatizer()

In [2]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/rahul/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
# df1 = pd.read_csv("/data/unified_parser_text_to_sql/create_database_ours/data/dataset details.xlsx - sample questions.csv")
df1 = pd.read_csv("/DATA1/rahul/text2SQL/booksql_generation/generated_data/dataset details.xlsx - sample questions.csv")

df1= df1[:183]
df1.head()

,question,sql,Levels,Changes
0,How much did [customer_name] spend on [account...,select sum(debit) from master_txn_table as T1...,hard,criteria : -\n\nEASY : simple queries with sin...
1,Did we get paid for the [product_name] in [dat...,"select distinct transaction_id, AR_paid from m...",medium,NaN
2,Have we billed [customer_name] for the [produc...,select distinct transaction_id from master_txn...,medium,NaN
3,What is the average spend per month on [accoun...,select sum(debit)/12 from master_txn_table as...,hard,NaN
4,what was our top [number] selling products and...,"select product_service, sum(quantity) from mas...",hard,add template here - done \n[number] can take v...


In [5]:
df1.tail()

,question,sql,Levels,Changes
178,Show the invoice number and the number of item...,"SELECT transaction_id , sum(quantity) FROM ma...",hard,Check this as well\n[SH] updated query
179,"Show the minimum, average, maximum order quant...","SELECT min(quantity_per_transaction) , avg(qu...",hard,Check this as well\n[SH] updated query
180,What are the names of products that have never...,SELECT product_service FROM Products_Service E...,medium,Check this as well\n[SH] looks good
181,Show all product types and the total quantity ...,"SELECT T2.product_service_type , sum(T1.quant...",hard,Check this as well\n[SH] updated query
182,How many products are never sold with total va...,SELECT count(*) FROM Product_Service WHERE pro...,hard,Check this as well\n[SH] updated query


In [6]:
# pip install openpyxl

In [7]:
df1['Levels'].value_counts()

Levels
medium    100
hard       80
easy        3
Name: count, dtype: int64

In [8]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 183 entries, 0 to 182
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   question  183 non-null    object
 1   sql       183 non-null    object
 2   Levels    183 non-null    object
 3   Changes   37 non-null     object
dtypes: object(4)
memory usage: 5.8+ KB


In [9]:
df1.rename(columns = {'question':'Query Pattern','sql':'Sql pattern'}, inplace = True)

In [18]:

def get_split(df):
    X_train, X_test, _, _ = train_test_split(df["Query Pattern"], df["Sql pattern"], test_size = 0.1, stratify=df[['Levels']])

    df.loc[X_train.index, "split"] = "train"
    df.loc[X_test.index, "split"] = "test"

    train_df = df[df['split'] == 'train']
    test_df = df[df['split'] == 'test']
    X_train, X_dev, _, _ = train_test_split(train_df["Query Pattern"], train_df["Sql pattern"], test_size = 0.1, stratify=train_df[['Levels']])

    train_df.loc[X_train.index, "split"] = "train"
    train_df.loc[X_dev.index, "split"] = "dev"

    df = pd.concat([train_df, test_df])
    return df

for fold_id in range(3):
    print("Fold: ", fold_id)
    df = df1.copy()
    df = get_split(df)
    if fold_id==0:
        print("Train split: \n", df[df['split'] == 'train']['Levels'].value_counts())
        print("Dev split: \n", df[df['split'] == 'dev']['Levels'].value_counts())
        print("Test split:  \n", df[df['split'] == 'test']['Levels'].value_counts())
    df.to_csv(f'/data/unified_parser_text_to_sql/create_database_ours/booksql/folds/fold_{fold_id}.csv')

Fold:  0
Train split: 
 medium    131
hard       19
easy        3
Name: Levels, dtype: int64
Dev split: 
 medium    15
hard       2
easy       1
Name: Levels, dtype: int64
Test split:  
 medium    16
hard       2
easy       1
Name: Levels, dtype: int64
Fold:  1
Fold:  2


/data/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/data/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/data/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versu

In [19]:
print("Train split: \n", df[df['split'] == 'train']['Levels'].value_counts())
print("Dev split: \n", df[df['split'] == 'dev']['Levels'].value_counts())
print("Test split:  \n", df[df['split'] == 'test']['Levels'].value_counts())

Train split: 
 medium    131
hard       19
easy        3
Name: Levels, dtype: int64
Dev split: 
 medium    15
hard       2
easy       1
Name: Levels, dtype: int64
Test split:  
 medium    16
hard       2
easy       1
Name: Levels, dtype: int64


In [20]:
customers = pd.read_csv("/data/unified_parser_text_to_sql/create_database_ours/booksql/customers.csv")
products = pd.read_csv("/data/unified_parser_text_to_sql/create_database_ours/booksql/products.csv")
vendors = pd.read_csv("/data/unified_parser_text_to_sql/create_database_ours/booksql/vendors.csv")
accounts = pd.read_csv("/data/unified_parser_text_to_sql/create_database_ours/booksql/chart_of_accounts.csv")
employee = pd.read_csv("/data/unified_parser_text_to_sql/create_database_ours/booksql/employees.csv")
date_ranges = pd.read_csv("/data/unified_parser_text_to_sql/create_database_ours/data/dataset details.xlsx - sample date ranges.csv")
date_ranges["sql"] = date_ranges["sql"].replace(r"\n","",regex=True)

ignore_keys = ["sample", "test", "quickbooks", "unknown", "my company", "customer", "cash", "sale", "deposit", ".", "payroll", "anonymous"]

In [21]:
print("customers shape: ", customers.shape)
print("products shape: ", products.shape)
print("vendors shape: ", vendors.shape)
print("accounts shape: ", accounts.shape)
print("employee shape: ", employee.shape)
print("date_ranges shape: ", date_ranges.shape)


customers shape:  (10000, 13)
products shape:  (270, 4)
vendors shape:  (10000, 8)
accounts shape:  (2430, 5)
employee shape:  (6000, 7)
date_ranges shape:  (25, 2)


In [22]:
date_ranges.head()

,sample date ranges in questions,sql
0,This fiscal year to date,"BETWEEN date(current_date, '-3 months', 'start..."
1,This year to date,"BETWEEN date(current_date, 'start of year') AN..."
2,This fiscal year,"BETWEEN date(current_date, '-3 months', 'start..."
3,This year,"BETWEEN date(current_date, 'start of year') AN..."
4,Last fiscal year,"BETWEEN date(current_date, '-3 months', 'start..."


In [23]:
def clean_names(df, var):
    flag = df[var].apply(lambda x: sum([y in x for y in ignore_keys]))
    df = df[flag == 0].reset_index(drop = True)
    return df

In [24]:
def stem_tokens(x):
    return " ".join([lemmatizer.lemmatize(y) for y in x.lower().split(" ")])

In [25]:
def get_variants(x):
    variants = [x.lower()]
    variants.append(stem_tokens(x))
    if "&" in x:
        x0 = x.split("&")[0].strip().lower()
        x1 = x.split("&")[1].strip().lower()
        variants.append(x0)
        variants.append(stem_tokens(x0))
        variants.append(x1)
        variants.append(stem_tokens(x1))
    if "|" in x:
        x0 = x.split("|")[0].strip().lower()
        x1 = x.split("|")[1].strip().lower()
        variants.append(x0)
        variants.append(stem_tokens(x0))
        variants.append(x1)
        variants.append(stem_tokens(x1))
    variants = np.unique(variants)
    return variants

In [26]:
def choose_random_attribute(filter_by_attribute):
    return random.sample(filter_by_attribute[random.sample(sorted(filter_by_attribute), 1)[0]].tolist(), 1)[0]

In [27]:
customers = clean_names(customers, "Customer name")
products = clean_names(products, "Product_Service")
vendors = clean_names(vendors, "Vendor name")
accounts = clean_names(accounts, "Account name")
employee = clean_names(employee, "Employee name")

In [28]:
customers["Variants"] = customers["Customer name"].apply(lambda x: get_variants(x))
products["Variants"] = products["Product_Service"].apply(lambda x: get_variants(x))
vendors["Variants"] = vendors["Vendor name"].apply(lambda x: get_variants(x))
accounts["Variants"] = accounts["Account name"].apply(lambda x: get_variants(x))
employee["Variants"] = employee["Employee name"].apply(lambda x: get_variants(x))
print("completed")

completed


In [29]:
# K = 50

aggregation_entity = ["total", "average", "mean", "max", "min", "first", "last", "highest", "lowest"]

intents = ["sales", "expense", "invoice", "bill", "account payable", "account receivable"]   #txn_type,

date_filter_entity = date_ranges["sample date ranges in questions"].tolist()
date_sql_entity = dict(zip(date_ranges["sample date ranges in questions"], date_ranges["sql"]))

filter_by_customer = customers.set_index("Customer name").to_dict()["Variants"]

filter_by_product = products.set_index("Product_Service").to_dict()["Variants"]

filter_by_vendor = vendors.set_index("Vendor name").to_dict()["Variants"]          #salesperson_name

filter_by_account = accounts.set_index("Account name").to_dict()["Variants"]

filter_by_employee = employee.set_index("Employee name").to_dict()["Variants"]

group_by_entity = ["account", "department", "vendor", "customer", "product"]



In [30]:
# attributes_list = df["Query Pattern"].apply(lambda x:  re.findall(r'\[.*?\]', str(x) ))
# attributes_list = attributes_list.apply(lambda x: [y.replace("[", "").replace("]", "") for y in x])
# attributes_list = [x for y in attributes_list for x in y]
# pd.Series(attributes_list).value_counts()

In [31]:
attributes_dict = {
    "date_filter": date_filter_entity,
    "customer_name": filter_by_customer,
    "vendor_name": filter_by_vendor,
    "product_name": filter_by_product,
    "aggregation_entity": aggregation_entity,
    "account_name": filter_by_account,
    "employee_name": filter_by_employee,
    "groupby_entity": group_by_entity,
    #
    # "txn_type": intents,
    "transaction_type":intents,
#     "salesperson_name": filter_by_vendor
}

In [32]:
# attributes_dict

In [33]:
def get_aggregation(query):
    if any(word in query for word in ["average", "mean"]):
        return "avg"
    elif any(word in query for word in ["max", "last", "highest"]):
        return "max"
    elif any(word in query for word in ["min", "first", "lowest"]):
        return "min"
    else:
        return "sum"

In [34]:
def choose_random_attribute(attribute_dict):
    # print("attribute_dict: ", attribute_dict)
    return random.sample(attribute_dict[random.sample(sorted(attribute_dict), 1)[0]].tolist(), 1)[0]

In [35]:
def f1(query_pattern,d_word,is_sql=0):
    attributes = re.findall(r'\[.*?\]', str(query_pattern))
    
#     print("att: ",attributes)
#     print("word: ",word)
    
#     assert len(attributes) == len(word)

#     print("asdasdasdas      ",d_word)
#     print("its attr : " , attributes)
#     print()
#     return ""
    
    

    for a in attributes:
        
        a_cleaned = re.sub(r"[\[\]]", "", a)

        new_word = d_word[0][a_cleaned]

        if is_sql == 1 and a_cleaned == "aggregation_entity" :
            new_word = get_aggregation(new_word)
        
        elif is_sql == 1 and a_cleaned == "date_filter":                          #"entity" above dict
            new_word = date_sql_entity[new_word]
            a = '"' + a + '"'
            
            
            # print("new word: ",new_word)
            # print("new_word: ", d_word[0])
            # # exit()
            # new_word = new_word.values



        try:
            query_pattern = query_pattern.replace(a, new_word)
        except:
            pass
            
        
#     for a,w in zip(attributes,word) :
#         a_cleaned = re.sub(r"[\[\]]", "", a)
#         query_pattern = query_pattern.replace(a, d_word[a])
# #     print("$$$$",query_pattern)
    return query_pattern

In [36]:

def replace_attributes(query_pattern):
    temp = []
    d = {}
    attributes = re.findall(r'\[.*?\]', str(query_pattern))
    for a in attributes:
        a_cleaned = re.sub(r"[\[\]]", "", a)
        
        if type(attributes_dict[a_cleaned]) == list:
            select_attribute = random.sample(attributes_dict[a_cleaned], 1)[0]
        else:
            select_attribute = choose_random_attribute(attributes_dict[a_cleaned])
            # print("select_attribute: ", select_attribute)

#         query_pattern = query_pattern.replace(a, select_attribute)
#         temp.append(select_attribute)
        
        d[a_cleaned]  =  select_attribute
    
    temp.append(d)
    # print("temp: ", temp)
    return temp

In [37]:
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

/data/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  This is separate from the ipykernel package so we can avoid doing imports until


In [38]:
# df.iloc[0]["Query Pattern"]

In [39]:
def simulate_questions(n, df):
    df_sample = df[~df["Query Pattern"].apply(lambda x: "salesperson_name" in x)].reset_index(drop=True)
    
    df_simulated = df_sample.loc[random.choices(range(len(df_sample)), k=n)].reset_index(drop=True)
    
#     df_simulated["Query"] = df_simulated["Query Pattern"].apply(lambda x: replace_attributes(x))
    
#     df_simulated["Sql pattern new"] = df_simulated["Sql pattern"].apply(lambda x: replace_attributes(str(x)))
    # df_simulated.loc[df_simulated["Aggregation"].isna(), "Aggregation"] = df_simulated[df_simulated["Aggregation"].isna()]["Query"].apply(lambda x: get_aggregation(x))

    ls1 = df_simulated["Query Pattern"].apply(lambda x: replace_attributes(x)).to_list()
    # print("ls1: ", ls1)
    df_simulated["Query"] = np.nan
    df_simulated["Sql pattern new"] = np.nan
    
    for i in range(len(ls1)):
        df_simulated.loc[[i],["Query"] ] = f1(df_simulated.iloc[i]["Query Pattern"],ls1[i])
        df_simulated.loc[[i],["Sql pattern new"] ] = f1(df_simulated.iloc[i]["Sql pattern"],ls1[i],is_sql=1)
    
    # df_simulated["Sql pattern new""].apply(lambda x : )
    
    return df_simulated

In [40]:
for fold_id in range(3):
    print("Fold id: ", fold_id)
    df = pd.read_csv(f"/data/unified_parser_text_to_sql/create_database_ours/booksql/folds/fold_{fold_id}.csv")
    df_simulated = simulate_questions(13000, df)
    df_simulated["Sql pattern"] = df_simulated["Sql pattern"].replace(r"\n","",regex=True)
    df_simulated["Sql pattern new"] = df_simulated["Sql pattern new"].replace(r"\n","",regex=True)
    final_df = df_simulated[['Query','Sql pattern new', 'Levels', 'split']]
    final_df.to_csv(f"/data/unified_parser_text_to_sql/create_database_ours/booksql/sql_pairs/sql_pairs_fold_id{fold_id}.csv")
    final_df.head()
print("Completed")

Fold id:  0
Fold id:  1
Fold id:  2
Completed
